# Yield simulations of itaconate from different substrates

Simulations are performed to identify the maximum yield for itaconate from various substrates.

## System libraries

In [1]:
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from cobra import Model, Reaction, Metabolite
from cobra.io import read_sbml_model

from escher import Builder

## Loading Model

In [2]:
ModelFile = join('..','model','iCL1079.xml')
model=read_sbml_model(ModelFile)

### Finding itaconate

In [3]:
# model.reactions.query('EX_f')
# model.reactions.EX_itacon_e
# model.metabolites.for_e
model.add_boundary(model.metabolites.for_e)

Reaction identifier,EX_for_e
Name,formate exchange
Memory address,0x07fe21cf540a0
Stoichiometry,for_e <=> formate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


## Testing substrates

In [4]:
Substrate_List = ['EX_glc__D_e','EX_ac_e','EX_for_e']
Sol_Dict = dict.fromkeys(Substrate_List)
Yield_Dict = dict.fromkeys(Substrate_List)
for Substrate in Substrate_List:
    with model:
        medium=model.medium
        medium['EX_glc__D_e']=0
        medium[Substrate]=10
#         medium['EX_for_e'] = 1
        model.medium=medium
        model.objective = 'EX_itacon_e'
        solution=model.optimize()
#         print(model.summary())
        Sol_Dict[Substrate] = solution
        Sub_Name = model.reactions.get_by_id(Substrate).reaction.split()[0]
        try:
            Sub_Catom = int(model.metabolites.get_by_id(Sub_Name).formula[1])
        except ValueError:
            Sub_Catom = 1
        Yield_Dict[Substrate] = round(solution.objective_value,2)*5/(Sub_Catom*medium[Substrate])

print('Yield: ')
Yield_Dict

Yield: 


{'EX_glc__D_e': 0.8333333333333334,
 'EX_ac_e': 0.8324999999999999,
 'EX_for_e': 0.0}

In [5]:
# load U. maydis ESCHER map with Pentose phosphate pathway, Glycolysis

CarbCore = join('..','model','U.maydis_ESCHER_map_PPPW_Glycolysis_TCA.json')
builder = Builder(
    map_json=CarbCore,
    model = model, # 'iCL1079.json',
)
# Run FBA with the model and add the flux data to the map
solution = builder.model.optimize()
builder.reaction_data = Sol_Dict['EX_glc__D_e'].fluxes

builder

Builder(reaction_data={'ANTPPT': 0.0, 'ATNS': 0.0, 'TRANS-RXNTFS-54': 0.0, 'TRANS-RXNTFS-55-CPD-12575//CPD-125…